## Install CoLab Tools


In [85]:
pip install tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Preprocessing

In [86]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [87]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [88]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [89]:
# Look at status value counts
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [90]:
# Look at special_considerations value counts
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [91]:
# Look at USE CASE value counts
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [92]:
# Look at organization value counts
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [93]:
# Drop the ORGANIZATION column
application_df.drop(columns=['ORGANIZATION'], inplace=True)

In [94]:
# Drop the STATUS column
application_df.drop(columns=['STATUS'], inplace=True)

In [95]:
# Drop the SPECIAL CONSIDERATIONS column
application_df.drop(columns=['SPECIAL_CONSIDERATIONS'], inplace=True)

In [96]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [97]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
other_app_types_df = pd.DataFrame(application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x<20])
application_types_to_replace = list(other_app_types_df.index.values)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       27
T12         27
Name: APPLICATION_TYPE, dtype: int64

In [98]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [99]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [100]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
other_classifcations_df = pd.DataFrame(application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<20])
classifications_to_replace = list(other_classifcations_df.index.values)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
Other      207
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
Name: CLASSIFICATION, dtype: int64

In [101]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
34295,5000,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
34296,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,5000,1,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,0,0,0


In [102]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [103]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [104]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=70, activation="relu", input_dim=55))

# More hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_49 (Dense)            (None, 70)                3920      
                                                                 
 dense_50 (Dense)            (None, 50)                3550      
                                                                 
 dense_51 (Dense)            (None, 40)                2040      
                                                                 
 dense_52 (Dense)            (None, 30)                1230      
                                                                 
 dense_53 (Dense)            (None, 20)                620       
                                                                 
 dense_54 (Dense)            (None, 10)                210       
                                                                 
 dense_55 (Dense)            (None, 20)               

In [105]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [106]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5771 - accuracy: 0.7162
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5602 - accuracy: 0.7276
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5588 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5571 - accuracy: 0.7290
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5559 - accuracy: 0.7289
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5555 - accuracy: 0.7302
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5544 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5543 - accuracy: 0.7295
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7312
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accura

In [109]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5754 - accuracy: 0.7269 - 255ms/epoch - 953us/step
Loss: 0.5754076838493347, Accuracy: 0.7268804907798767


In [110]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharityOptimized2.h5')